In [ ]:
import mlflow
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import logging
import numpy as np

import re
import os
import json
from typing import List, Dict

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class SentimentInferencer:
    def __init__(self):
        self.label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
        self.inverse_mapping = {v: k for k, v in self.label_mapping.items()}
        self.model = None

    def load_model(self, model_name="randomforest"):
        """Carrega o modelo do MLflow."""
        try:
            model_uri = f"models:/{model_name}/latest"
            self.model = mlflow.sklearn.load_model(model_uri)
            logger.info(f"Modelo '{model_name}' carregado com sucesso de MLflow.")
        except Exception as e:
            logger.error(f"Erro ao carregar o modelo '{model_name}' de MLflow: {e}")
            raise

    def preprocess_data(self, data):
        """Pré-processa os dados de entrada para inferência.
           Se o seu modelo no MLflow já inclui o vetorizador,
           esta etapa pode ser mais simples ou até mesmo desnecessária."""
        if self.model is None:
            raise ValueError("Modelo não carregado. Chame load_model() primeiro.")

        # Se o modelo carregado já é um pipeline que inclui o vetorizador,
        # você pode simplesmente passar os dados de texto brutos para o predict.
        # Caso contrário, se você precisar aplicar o mesmo vetorizador usado no treinamento:
        #
        # # Tentar acessar o vetorizador do pipeline (se o modelo for um pipeline)
        # if hasattr(self.model, 'steps'):
        #     for name, step in self.model.steps:
        #         if isinstance(step, TfidfVectorizer):
        #             return step.transform(data)
        #
        # # Se o vetorizador foi salvo separadamente (o que não parece ser o caso no seu código de validação)
        # # você precisaria carregá-lo aqui e usar self.vectorizer.transform(data)
        #
        # # Assumindo que o modelo já lida com a transformação:
        return data

    def predict_sentiment(self, input_data):
        """Realiza a inferência de sentimento nos dados de entrada."""
        if self.model is None:
            raise ValueError("Modelo não carregado. Chame load_model() primeiro.")

        processed_data = self.preprocess_data(input_data)
        predictions = self.model.predict(processed_data)

        # Converter previsões numéricas de volta para rótulos de texto
        #predicted_labels = [self.inverse_mapping.get(int(pred), 'Unknown') for pred in predictions]
        return predictions


In [ ]:
# Configuração do MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")  # Ajuste para seu servidor MLflow

inferencer = SentimentInferencer()
inferencer.load_model(model_name="sentiment_randomforest")  # Substitua pelo nome do seu modelo
new_comments = []
for index, row in df.head(200).iterrows():
    if row['comment'] != row['comment_cleaned']:
        linha = index + 1  # Adiciona 1 para começar a contagem do 1
        new_comments.append(row['comment'])
        new_comments.append(row['comment_cleaned'])
    
        
# # Novos dados para inferência (lista de comentários)
# new_comments = [
#     "This restaurant is incredible, loved it!",
#     "Hmmm didn't like the experience... quite bad.",
#     "Avarage service, nothing special.",
#     "Cool... happy I am."
# ]

try:
    predictions = inferencer.predict_sentiment(new_comments)
    print("\nResultados da Inferência:")
    for comment, prediction in zip(new_comments, predictions):
        print(f"Comentário: '{comment}' -> Sentimento Previsto: '{prediction}'")

except ValueError as e:
    logger.error(f"Erro durante a inferência: {e}")
except Exception as e:
    logger.error(f"Ocorreu um erro inesperado: {e}")